In [17]:
import tensorflow as tf
import numpy as np

def diagnose_seq_encoding_issues(file_path, num_examples=2):
    """
    Diagnoses issues with seq_1hot encoding to understand why there are so many unknowns.
    """
    print(f"Diagnosing seq_1hot encoding issues in: {file_path}")
    dataset = tf.data.TFRecordDataset(file_path)
    
    amino_acids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 
                   'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
    
    for i, raw_record in enumerate(dataset.take(num_examples)):
        print(f"\n{'='*60}")
        print(f"DIAGNOSTIC FOR PROTEIN {i+1}")
        print('='*60)
        
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        
        prot_id = example.features.feature['prot_id'].bytes_list.value[0].decode('utf-8')
        L_value = list(example.features.feature['L'].int64_list.value)[0]
        seq_1hot = list(example.features.feature['seq_1hot'].float_list.value)
        
        print(f"Protein ID: {prot_id}")
        print(f"L (sequence length): {L_value}")
        print(f"seq_1hot total values: {len(seq_1hot)}")
        
        # Check the first 50 positions in detail
        print(f"\nDetailed analysis of first 50 positions:")
        print("Pos | Values Sum | Max Val | Has 1.0 | AA | Raw Values (first 5)")
        print("-" * 80)
        
        valid_positions = 0
        zero_positions = 0
        multi_one_positions = 0
        
        for pos in range(min(50, len(seq_1hot) // 20)):
            start_idx = pos * 20
            end_idx = start_idx + 20
            pos_values = seq_1hot[start_idx:end_idx]
            
            # Statistics for this position
            values_sum = sum(pos_values)
            max_val = max(pos_values)
            ones_count = sum(1 for v in pos_values if v == 1.0)
            has_one = ones_count > 0
            
            # Determine amino acid
            if ones_count == 1:
                aa_idx = pos_values.index(1.0)
                aa = amino_acids[aa_idx]
                valid_positions += 1
            elif ones_count == 0:
                aa = "NONE"
                zero_positions += 1
            else:
                aa = f"MULTI({ones_count})"
                multi_one_positions += 1
            
            # Show first 5 values for inspection
            first_5 = [f"{v:.1f}" for v in pos_values[:5]]
            
            print(f"{pos+1:3d} | {values_sum:10.1f} | {max_val:7.1f} | {has_one:7} | {aa:4s} | {first_5}")
        
        print(f"\nSummary for first 50 positions:")
        print(f"  - Valid amino acid positions: {valid_positions}")
        print(f"  - Zero positions (no amino acid): {zero_positions}")
        print(f"  - Multi-1.0 positions: {multi_one_positions}")
        
        # Check if there's a pattern in the encoding
        print(f"\nChecking for encoding patterns:")
        
        # Look for non-standard values
        unique_values = set(seq_1hot)
        print(f"  - Unique values in seq_1hot: {sorted(unique_values)}")
        
        # Check if values are exactly 0.0 and 1.0
        non_binary = [v for v in seq_1hot if v != 0.0 and v != 1.0]
        if non_binary:
            print(f"  - Non-binary values found: {len(non_binary)} values")
            print(f"  - Sample non-binary values: {non_binary[:10]}")
        
        # Check the total number of 1.0s across all positions
        total_ones = sum(1 for v in seq_1hot if v == 1.0)
        print(f"  - Total 1.0 values: {total_ones}")
        print(f"  - Expected (L): {L_value}")
        print(f"  - Ratio: {total_ones/L_value:.3f}")
        
        # Look at the distribution of 1.0s across positions
        ones_per_position = []
        for pos in range(len(seq_1hot) // 20):
            start_idx = pos * 20
            end_idx = start_idx + 20
            pos_values = seq_1hot[start_idx:end_idx]
            ones_count = sum(1 for v in pos_values if v == 1.0)
            ones_per_position.append(ones_count)
        
        # Count positions by number of 1.0s
        from collections import Counter
        ones_distribution = Counter(ones_per_position)
        print(f"\nDistribution of 1.0s per position:")
        for num_ones, count in sorted(ones_distribution.items()):
            print(f"  - {num_ones} ones: {count} positions")
        
        # Check if there's an offset or different indexing
        print(f"\nChecking for potential indexing issues:")
        
        # Look for the first valid amino acid
        first_valid_pos = None
        for pos in range(min(20, len(seq_1hot) // 20)):
            start_idx = pos * 20
            end_idx = start_idx + 20
            pos_values = seq_1hot[start_idx:end_idx]
            if sum(1 for v in pos_values if v == 1.0) == 1:
                first_valid_pos = pos
                break
        
        if first_valid_pos is not None:
            print(f"  - First valid amino acid at position: {first_valid_pos + 1}")
        else:
            print(f"  - No valid amino acids found in first 20 positions!")
        
        # Check if the sequence might be 0-indexed vs 1-indexed
        print(f"\nTesting different interpretations:")
        
        # Try interpreting as 0-indexed (skip position 0)
        valid_from_pos_0 = 0
        for pos in range(1, min(21, len(seq_1hot) // 20)):
            start_idx = pos * 20
            end_idx = start_idx + 20
            pos_values = seq_1hot[start_idx:end_idx]
            if sum(1 for v in pos_values if v == 1.0) == 1:
                valid_from_pos_0 += 1
        
        print(f"  - Valid amino acids in positions 1-20 (0-indexed): {valid_from_pos_0}")

# Run the diagnostic
train_file = "PDB-GO/PDB_GO_train_00-of-30.tfrecords"
diagnose_seq_encoding_issues(train_file)

Diagnosing seq_1hot encoding issues in: PDB-GO/PDB_GO_train_00-of-30.tfrecords

DIAGNOSTIC FOR PROTEIN 1
Protein ID: 2ZPA-A
L (sequence length): 671
seq_1hot total values: 17446

Detailed analysis of first 50 positions:
Pos | Values Sum | Max Val | Has 1.0 | AA | Raw Values (first 5)
--------------------------------------------------------------------------------
  1 |        0.0 |     0.0 |       0 | NONE | ['0.0', '0.0', '0.0', '0.0', '0.0']
  2 |        1.0 |     1.0 |       1 | Q    | ['0.0', '0.0', '0.0', '0.0', '0.0']
  3 |        1.0 |     1.0 |       1 | G    | ['0.0', '0.0', '0.0', '0.0', '0.0']
  4 |        1.0 |     1.0 |       1 | I    | ['0.0', '0.0', '0.0', '0.0', '0.0']
  5 |        1.0 |     1.0 |       1 | N    | ['0.0', '0.0', '1.0', '0.0', '0.0']
  6 |        1.0 |     1.0 |       1 | L    | ['0.0', '0.0', '0.0', '0.0', '0.0']
  7 |        0.0 |     0.0 |       0 | NONE | ['0.0', '0.0', '0.0', '0.0', '0.0']
  8 |        1.0 |     1.0 |       1 | K    | ['0.0', '0.0',

In [12]:
import tensorflow as tf
import numpy as np

def inspect_protein_labels(file_path, num_examples=2):
    """
    Inspects a protein record and its labels from a TFRecord file.
    For each label type (cc, mf, bp), identifies which positions have a value of 1.
    """
    print(f"Inspecting file: {file_path}")
    dataset = tf.data.TFRecordDataset(file_path)
    
    for i, raw_record in enumerate(dataset.take(num_examples)):
        print(f"\nProtein {i+1}:")
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        
        # Get protein ID
        prot_id = example.features.feature['prot_id'].bytes_list.value[0].decode('utf-8')
        print(f"Protein ID: {prot_id}")
        
        # Print shape information for all features
        print("\nFeature shapes:")
        for key in example.features.feature:
            feature = example.features.feature[key]
            
            if feature.HasField('bytes_list'):
                shape = len(feature.bytes_list.value)
                dtype = "bytes"
            elif feature.HasField('float_list'):
                shape = len(feature.float_list.value)
                dtype = "float"
            elif feature.HasField('int64_list'):
                shape = len(feature.int64_list.value)
                dtype = "int64"
            
            print(f"  - {key}: {dtype}, shape={shape}")
        
        # Extract and analyze label features
        label_types = {
            'cc_labels': 'Cellular Component',
            'mf_labels': 'Molecular Function',
            'bp_labels': 'Biological Process'
        }
        
        print("\nLabel analysis:")
        for label_key, label_name in label_types.items():
            if label_key in example.features.feature:
                labels = list(example.features.feature[label_key].int64_list.value)
                active_indices = [i for i, val in enumerate(labels) if val == 1]
                
                print(f"\n{label_name} ({label_key}):")
                print(f"  - Total labels: {len(labels)}")
                print(f"  - Active labels (value=1): {len(active_indices)}")
                if len(active_indices) > 0:
                    print(f"  - Active indices: {active_indices}")
                    
        # Examine sequence one-hot encoding if available
        if 'seq_1hot' in example.features.feature:
            seq_1hot = list(example.features.feature['seq_1hot'].float_list.value)
            seq_length = len(seq_1hot) // 20  # Assuming 20 amino acids
            
            if seq_length * 20 == len(seq_1hot):
                print(f"\nSequence one-hot encoding:")
                print(f"  - Sequence length: {seq_length}")
                print(f"  - Total one-hot values: {len(seq_1hot)}")
                
                # Extract a small sample of the sequence (first 5 positions)
                if seq_length >= 5:
                    print("  - First 5 positions (20 amino acids per position):")
                    for pos in range(5):
                        pos_values = seq_1hot[pos*20:(pos+1)*20]
                        if 1.0 in pos_values:
                            aa_index = pos_values.index(1.0)
                            print(f"    Position {pos+1}: 1.0 at index {aa_index}")
                        else:
                            print(f"    Position {pos+1}: No 1.0 value found")

# Use proper file path with forward slashes
train_file = "PDB-GO/PDB_GO_train_00-of-30.tfrecords"
inspect_protein_labels(train_file)

Inspecting file: PDB-GO/PDB_GO_train_00-of-30.tfrecords

Protein 1:
Protein ID: 2ZPA-A

Feature shapes:
  - ca_dist_matrix: float, shape=450241
  - bp_labels: int64, shape=1943
  - mf_labels: int64, shape=489
  - cb_dist_matrix: float, shape=450241
  - seq_1hot: float, shape=17446
  - L: int64, shape=1
  - prot_id: bytes, shape=1
  - cc_labels: int64, shape=320

Label analysis:

Cellular Component (cc_labels):
  - Total labels: 320
  - Active labels (value=1): 0

Molecular Function (mf_labels):
  - Total labels: 489
  - Active labels (value=1): 15
  - Active indices: [9, 68, 74, 88, 98, 114, 183, 205, 253, 270, 337, 339, 463, 482, 488]

Biological Process (bp_labels):
  - Total labels: 1943
  - Active labels (value=1): 10
  - Active indices: [46, 136, 192, 273, 322, 623, 637, 781, 1221, 1417]

Protein 2:
Protein ID: 2GBB-A

Feature shapes:
  - ca_dist_matrix: float, shape=24336
  - L: int64, shape=1
  - mf_labels: int64, shape=489
  - prot_id: bytes, shape=1
  - cc_labels: int64, shape

In [15]:
import tensorflow as tf
import numpy as np

def inspect_seq_and_L_values(file_path, num_examples=2):
    """
    Inspects seq_1hot and L values from a TFRecord file.
    Prints the actual values for L and detailed seq_1hot information.
    """
    print(f"Inspecting file: {file_path}")
    dataset = tf.data.TFRecordDataset(file_path)
    
    for i, raw_record in enumerate(dataset.take(num_examples)):
        print(f"\n{'='*50}")
        print(f"Protein {i+1}:")
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        
        # Get protein ID
        prot_id = example.features.feature['prot_id'].bytes_list.value[0].decode('utf-8')
        print(f"Protein ID: {prot_id}")
        
        # Extract and print L value
        if 'L' in example.features.feature:
            L_value = list(example.features.feature['L'].int64_list.value)[0]
            print(f"\nL (sequence length): {L_value}")
        
        # Extract and analyze seq_1hot
        if 'seq_1hot' in example.features.feature:
            seq_1hot = list(example.features.feature['seq_1hot'].float_list.value)
            seq_length = len(seq_1hot) // 20  # 20 amino acids per position
            
            print(f"\nseq_1hot analysis:")
            print(f"  - Total values in seq_1hot: {len(seq_1hot)}")
            print(f"  - Calculated sequence length: {seq_length}")
            print(f"  - Expected total (seq_length * 20): {seq_length * 20}")
            
            # Print first 10 positions of the sequence
            print(f"\nFirst 10 amino acid positions:")
            amino_acids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 
                          'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
            
            for pos in range(min(10, seq_length)):
                pos_values = seq_1hot[pos*20:(pos+1)*20]
                # Find which amino acid is encoded (should have value 1.0)
                for aa_idx, val in enumerate(pos_values):
                    if val == 1.0:
                        print(f"  Position {pos+1}: {amino_acids[aa_idx]} (index {aa_idx})")
                        break
                else:
                    print(f"  Position {pos+1}: No amino acid found (all zeros)")
            
            # Print some statistics about the one-hot encoding
            total_ones = sum(1 for val in seq_1hot if val == 1.0)
            total_zeros = sum(1 for val in seq_1hot if val == 0.0)
            other_values = len(seq_1hot) - total_ones - total_zeros
            
            print(f"\nseq_1hot statistics:")
            print(f"  - Total 1.0 values: {total_ones}")
            print(f"  - Total 0.0 values: {total_zeros}")
            print(f"  - Other values: {other_values}")
            
            # Show a sample of the raw values (first 40 values = first 2 positions)
            print(f"\nRaw seq_1hot values (first 40 = first 2 positions):")
            for i in range(0, min(40, len(seq_1hot)), 20):
                position_vals = seq_1hot[i:i+20]
                print(f"  Position {i//20 + 1}: {position_vals}")

# Use the function
train_file = "PDB-GO/PDB_GO_train_00-of-30.tfrecords"
inspect_seq_and_L_values(train_file)

Inspecting file: PDB-GO/PDB_GO_train_00-of-30.tfrecords

Protein 1:
Protein ID: 2ZPA-A

L (sequence length): 671

seq_1hot analysis:
  - Total values in seq_1hot: 17446
  - Calculated sequence length: 872
  - Expected total (seq_length * 20): 17440

First 10 amino acid positions:
  Position 1: No amino acid found (all zeros)
  Position 2: Q (index 5)
  Position 3: G (index 7)
  Position 4: I (index 9)
  Position 5: N (index 2)
  Position 6: L (index 10)
  Position 7: No amino acid found (all zeros)
  Position 8: K (index 11)
  Position 9: A (index 0)
  Position 10: L (index 10)

seq_1hot statistics:
  - Total 1.0 values: 671
  - Total 0.0 values: 16775
  - Other values: 0

Raw seq_1hot values (first 40 = first 2 positions):
  Position 1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Position 2: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Protein 2:
Protein ID: 2GBB-A

L (s

In [4]:
import tensorflow as tf
import numpy as np

# ------------------------------------------------------------------
#  Add this helper just once, outside the loop
AA_TABLE = "ACDEFGHIKLMNPQRSTVWYXBZJUO"   # 26-way alphabet
AA_COLS  = len(AA_TABLE)                  # 26
AA_COLS20 = 20                            # canonical set
# ------------------------------------------------------------------

def inspect_protein_labels(file_path, num_examples=2):
    """
    Inspects a protein record and its labels from a TFRecord file.
    For each label type (cc, mf, bp), identifies which positions have a value of 1.
    """
    print(f"Inspecting file: {file_path}")
    dataset = tf.data.TFRecordDataset(file_path)
    
    for i, raw_record in enumerate(dataset.take(num_examples)):
        print(f"\nProtein {i+1}:")
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        
        # Get protein ID
        prot_id = example.features.feature['prot_id'].bytes_list.value[0].decode('utf-8')
        print(f"Protein ID: {prot_id}")
        
        # Print shape information for all features
        print("\nFeature shapes:")
        for key in example.features.feature:
            feature = example.features.feature[key]
            
            if feature.HasField('bytes_list'):
                shape = len(feature.bytes_list.value)
                dtype = "bytes"
            elif feature.HasField('float_list'):
                shape = len(feature.float_list.value)
                dtype = "float"
            elif feature.HasField('int64_list'):
                shape = len(feature.int64_list.value)
                dtype = "int64"
            
            print(f"  - {key}: {dtype}, shape={shape}")
        
        # Extract and analyze label features
        label_types = {
            'cc_labels': 'Cellular Component',
            'mf_labels': 'Molecular Function',
            'bp_labels': 'Biological Process'
        }
        
        print("\nLabel analysis:")
        for label_key, label_name in label_types.items():
            if label_key in example.features.feature:
                labels = list(example.features.feature[label_key].int64_list.value)
                active_indices = [i for i, val in enumerate(labels) if val == 1]
                
                print(f"\n{label_name} ({label_key}):")
                print(f"  - Total labels: {len(labels)}")
                print(f"  - Active labels (value=1): {len(active_indices)}")
                if len(active_indices) > 0:
                    print(f"  - Active indices: {active_indices}")
        
        # ---------- NEW: decode and print amino-acid sequence ----------
        if 'seq_1hot' in example.features.feature:
            one_hot_flat = example.features.feature['seq_1hot'].float_list.value
            
            if len(one_hot_flat) % 26 == 0:
                L = len(one_hot_flat) // 26
                # NOTE the (26, L) shape  ↓  and that we take argmax over axis 0
                one_hot = np.asarray(one_hot_flat, dtype=np.float32).reshape(26, L)
                aa_idx  = one_hot.argmax(axis=0)          # length-L vector
                sequence = ''.join(AA_TABLE[i] for i in aa_idx)
                print(f"\nDecoded sequence (L={L}): {sequence[:60]}…")
            elif len(one_hot_flat) % 20 == 0:
                L = len(one_hot_flat) // 20
                one_hot = np.asarray(one_hot_flat, dtype=np.float32).reshape(20, L)
                aa_idx  = one_hot.argmax(axis=0)
                canon   = AA_TABLE[:20]                   # A-Y
                sequence = ''.join(canon[i] for i in aa_idx)
                print(f"\nDecoded 20-col sequence (L={L}): {sequence[:60]}…")
            else:
                print("\n⚠️ seq_1hot length not divisible by 20 or 26")

# Use proper file path with forward slashes
train_file = "PDB-GO/PDB_GO_train_00-of-30.tfrecords"
inspect_protein_labels(train_file)

Inspecting file: PDB-GO/PDB_GO_train_00-of-30.tfrecords

Protein 1:
Protein ID: 2ZPA-A

Feature shapes:
  - prot_id: bytes, shape=1
  - cb_dist_matrix: float, shape=450241
  - mf_labels: int64, shape=489
  - ca_dist_matrix: float, shape=450241
  - cc_labels: int64, shape=320
  - seq_1hot: float, shape=17446
  - L: int64, shape=1
  - bp_labels: int64, shape=1943

Label analysis:

Cellular Component (cc_labels):
  - Total labels: 320
  - Active labels (value=1): 0

Molecular Function (mf_labels):
  - Total labels: 489
  - Active labels (value=1): 15
  - Active indices: [9, 68, 74, 88, 98, 114, 183, 205, 253, 270, 337, 339, 463, 482, 488]

Biological Process (bp_labels):
  - Total labels: 1943
  - Active labels (value=1): 10
  - Active indices: [46, 136, 192, 273, 322, 623, 637, 781, 1221, 1417]

Decoded sequence (L=671): AVMIXQACAKAZAAQGEAKTAVSAFAAAAIRAPWDNAMAILOHUBAGACAFAPAAATGAA…

Protein 2:
Protein ID: 2GBB-A

Feature shapes:
  - seq_1hot: float, shape=4056
  - mf_labels: int64, shape

<h1>GO TERMS</h1>

In [2]:
import json, pathlib, tensorflow as tf

# path to the JSON that matches the ontology you are decoding
PARAMS = pathlib.Path(
r"C:\Users\rfrjo\Documents\Codebases\PFP_TESTING\trained_models\DeepFRI-MERGED_MultiGraphConv_3x512_fcd_1024_ca_10A_molecular_function_model_params.json")

with PARAMS.open() as fh:
    vocab = json.load(fh)

MF_IDS   = vocab["goterms"]   # ['GO:0000166', 'GO:0000287', …]  length 489
MF_NAMES = vocab["gonames"]   # ['nucleotide binding', 'magnesium-ion binding', …]

active_idx = [9, 68, 74, 88, 98, 114, 183, 205, 253, 270, 337, 339, 463, 482, 488]       # from your print-out

active_go  = [MF_IDS[i]   for i in active_idx]       # GO IDs
active_txt = [MF_NAMES[i] for i in active_idx]       # English names

for go_id, name in zip(active_go, active_txt):
    print(f"{go_id:<12}  {name}")


GO:0032559    adenyl ribonucleotide binding
GO:0008080    N-acetyltransferase activity
GO:0016746    transferase activity, transferring acyl groups
GO:0017076    purine nucleotide binding
GO:0005524    ATP binding
GO:0035639    purine ribonucleoside triphosphate binding
GO:0016410    N-acyltransferase activity
GO:0016747    transferase activity, transferring acyl groups other than amino-acyl groups
GO:0097367    carbohydrate derivative binding
GO:0000049    tRNA binding
GO:0032555    purine ribonucleotide binding
GO:0030554    adenyl nucleotide binding
GO:0003723    RNA binding
GO:0016407    acetyltransferase activity
GO:0032553    ribonucleotide binding


In [5]:
import json, pathlib, tensorflow as tf

# path to the JSON that matches the ontology you are decoding
PARAMS = pathlib.Path(
r"C:\Users\rfrjo\Documents\Codebases\PFP_Testing\trained_models\DeepCNN-MERGED_cellular_component_model_params.json")

with PARAMS.open() as fh:
    vocab = json.load(fh)

CC_IDS   = vocab["goterms"]   # ['GO:0000166', 'GO:0000287', …]  length 489
CC_NAMES = vocab["gonames"]   # ['nucleotide binding', 'magnesium-ion binding', …]

active_idx = []      # from your print-out

active_go  = [CC_IDS[i]   for i in active_idx]       # GO IDs
active_txt = [CC_NAMES[i] for i in active_idx]       # English names

for go_id, name in zip(active_go, active_txt):
    print(f"{go_id:<12}  {name}")


In [7]:
import json, pathlib, tensorflow as tf

# path to the JSON that matches the ontology you are decoding
PARAMS = pathlib.Path(
r"C:\Users\rfrjo\Documents\Codebases\PFP_Testing\trained_models\DeepCNN-MERGED_biological_process_model_params.json")

with PARAMS.open() as fh:
    vocab = json.load(fh)

BP_IDS   = vocab["goterms"]   # ['GO:0000166', 'GO:0000287', …]  length 489
BP_NAMES = vocab["gonames"]   # ['nucleotide binding', 'magnesium-ion binding', …]

active_idx = [46, 136, 192, 273, 322, 623, 637, 781, 1221, 1417] # from your print-out

active_go  = [BP_IDS[i]   for i in active_idx]       # GO IDs
active_txt = [BP_NAMES[i] for i in active_idx]       # English names

for go_id, name in zip(active_go, active_txt):
    print(f"{go_id:<12}  {name}")


GO:0008033    tRNA processing
GO:0006399    tRNA metabolic process
GO:0034660    ncRNA metabolic process
GO:0002097    tRNA wobble base modification
GO:0016070    RNA metabolic process
GO:0043412    macromolecule modification
GO:0006396    RNA processing
GO:0034470    ncRNA processing
GO:0006400    tRNA modification
GO:0009451    RNA modification


In [13]:
import os
from pathlib import Path
import re
from collections import defaultdict

def parse_fasta_protein_ids(fasta_path):
    """
    Parse FASTA file and extract protein IDs.
    Expected format: >{protein_id} nrPDB
    """
    protein_ids = set()
    
    try:
        with open(fasta_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('>'):
                    # Extract protein ID from header line
                    # Format: >{protein_id} nrPDB
                    parts = line.split()
                    if len(parts) >= 1:
                        protein_id = parts[0][1:]  # Remove the '>' character
                        protein_ids.add(protein_id)
    
    except FileNotFoundError:
        print(f"Error: FASTA file not found at {fasta_path}")
        return set()
    except Exception as e:
        print(f"Error reading FASTA file: {e}")
        return set()
    
    return protein_ids

def get_protein_data_ids(protein_data_dir):
    """
    Get all protein IDs from the protein_data_pdb directory structure.
    """
    protein_ids = set()
    
    if not os.path.exists(protein_data_dir):
        print(f"Error: Protein data directory not found at {protein_data_dir}")
        return set()
    
    try:
        for item in os.listdir(protein_data_dir):
            item_path = os.path.join(protein_data_dir, item)
            if os.path.isdir(item_path):
                protein_ids.add(item)
    except Exception as e:
        print(f"Error reading protein data directory: {e}")
        return set()
    
    return protein_ids

def analyze_sequence_coverage():
    """
    Main function to analyze coverage between protein structure data and FASTA sequences.
    """
    
    # Configuration
    fasta_path = r"C:\Users\rfrjo\Documents\Codebases\PFP_Testing\nrPDB-GO_2019.06.18_sequences.fasta"
    protein_data_dir = "protein_data_pdb"
    
    print("="*70)
    print("PROTEIN SEQUENCE COVERAGE ANALYSIS")
    print("="*70)
    
    # Parse FASTA file for protein IDs
    print(f"Parsing FASTA file: {fasta_path}")
    fasta_protein_ids = parse_fasta_protein_ids(fasta_path)
    print(f"Found {len(fasta_protein_ids)} protein sequences in FASTA file")
    
    # Get protein IDs from structure data
    print(f"\nScanning protein data directory: {protein_data_dir}")
    structure_protein_ids = get_protein_data_ids(protein_data_dir)
    print(f"Found {len(structure_protein_ids)} proteins in structure data")
    
    if len(fasta_protein_ids) == 0 or len(structure_protein_ids) == 0:
        print("Error: Could not load protein data. Please check file paths.")
        return
    
    # Find matches and mismatches
    matching_proteins = structure_protein_ids.intersection(fasta_protein_ids)
    missing_sequences = structure_protein_ids - fasta_protein_ids
    extra_sequences = fasta_protein_ids - structure_protein_ids
    
    # Calculate coverage statistics
    coverage_percentage = (len(matching_proteins) / len(structure_protein_ids)) * 100
    
    print("\n" + "="*70)
    print("COVERAGE ANALYSIS RESULTS")
    print("="*70)
    
    print(f"Structure proteins with sequences: {len(matching_proteins)}")
    print(f"Structure proteins missing sequences: {len(missing_sequences)}")
    print(f"Extra sequences (not in structure data): {len(extra_sequences)}")
    print(f"\nCOVERAGE PERCENTAGE: {coverage_percentage:.2f}%")
    
    # Detailed breakdown
    print(f"\nDetailed Breakdown:")
    print(f"  • Total proteins in structure data: {len(structure_protein_ids)}")
    print(f"  • Total sequences in FASTA: {len(fasta_protein_ids)}")
    print(f"  • Proteins with both structure & sequence: {len(matching_proteins)}")
    print(f"  • Proteins with structure but no sequence: {len(missing_sequences)}")
    print(f"  • Sequences without structure data: {len(extra_sequences)}")
    
    # Show some examples if there are missing sequences
    if len(missing_sequences) > 0:
        print(f"\nSample proteins missing sequences (showing first 10):")
        sample_missing = sorted(list(missing_sequences))[:10]
        for i, protein_id in enumerate(sample_missing, 1):
            print(f"  {i}. {protein_id}")
        if len(missing_sequences) > 10:
            print(f"  ... and {len(missing_sequences) - 10} more")
    
    # Show some examples of extra sequences
    if len(extra_sequences) > 0:
        print(f"\nSample sequences without structure data (showing first 10):")
        sample_extra = sorted(list(extra_sequences))[:10]
        for i, protein_id in enumerate(sample_extra, 1):
            print(f"  {i}. {protein_id}")
        if len(extra_sequences) > 10:
            print(f"  ... and {len(extra_sequences) - 10} more")
    
    # Show some examples of successful matches
    if len(matching_proteins) > 0:
        print(f"\nSample successful matches (showing first 10):")
        sample_matches = sorted(list(matching_proteins))[:10]
        for i, protein_id in enumerate(sample_matches, 1):
            print(f"  {i}. {protein_id}")
        if len(matching_proteins) > 10:
            print(f"  ... and {len(matching_proteins) - 10} more")
    
    print("\n" + "="*70)
    print("ANALYSIS COMPLETE")
    print("="*70)
    
    # Return results for potential further use
    return {
        'total_structure_proteins': len(structure_protein_ids),
        'total_fasta_sequences': len(fasta_protein_ids),
        'matching_proteins': len(matching_proteins),
        'missing_sequences': len(missing_sequences),
        'extra_sequences': len(extra_sequences),
        'coverage_percentage': coverage_percentage,
        'matching_protein_ids': matching_proteins,
        'missing_sequence_ids': missing_sequences,
        'extra_sequence_ids': extra_sequences
    }

def verify_sample_proteins(sample_size=5):
    """
    Verify a few sample proteins by checking their structure data files.
    """
    protein_data_dir = "protein_data_pdb"
    
    if not os.path.exists(protein_data_dir):
        print(f"Protein data directory not found: {protein_data_dir}")
        return
    
    print(f"\nVerifying sample protein data structure:")
    print("-" * 50)
    
    # Get a few sample proteins
    protein_dirs = [d for d in os.listdir(protein_data_dir) 
                   if os.path.isdir(os.path.join(protein_data_dir, d))]
    
    sample_proteins = protein_dirs[:sample_size]
    
    for protein_id in sample_proteins:
        protein_path = os.path.join(protein_data_dir, protein_id)
        files = os.listdir(protein_path)
        print(f"{protein_id}: {len(files)} files ({', '.join(files)})")

if __name__ == "__main__":
    # Run the coverage analysis
    results = analyze_sequence_coverage()
    
    # Verify sample protein structure
    verify_sample_proteins()
    
    print(f"\nScript completed successfully!")
    if results:
        print(f"Coverage Summary: {results['coverage_percentage']:.1f}% of structure proteins have sequences")
#NOTE: VAL ALSO HAVE SEQUENCE COVERAGE AT 100%

PROTEIN SEQUENCE COVERAGE ANALYSIS
Parsing FASTA file: C:\Users\rfrjo\Documents\Codebases\PFP_Testing\nrPDB-GO_2019.06.18_sequences.fasta
Found 36641 protein sequences in FASTA file

Scanning protein data directory: protein_data_pdb
Found 29740 proteins in structure data

COVERAGE ANALYSIS RESULTS
Structure proteins with sequences: 29740
Structure proteins missing sequences: 0
Extra sequences (not in structure data): 6901

COVERAGE PERCENTAGE: 100.00%

Detailed Breakdown:
  • Total proteins in structure data: 29740
  • Total sequences in FASTA: 36641
  • Proteins with both structure & sequence: 29740
  • Proteins with structure but no sequence: 0
  • Sequences without structure data: 6901

Sample sequences without structure data (showing first 10):
  1. 11AS-A
  2. 18GS-A
  3. 192L-A
  4. 1A0A-A
  5. 1A0P-A
  6. 1A21-A
  7. 1A22-A
  8. 1A27-A
  9. 1A3Q-A
  10. 1A44-A
  ... and 6891 more

Sample successful matches (showing first 10):
  1. 154L-A
  2. 155C-A
  3. 16PK-A
  4. 16VP-A
  5. 